In [1]:
import pandas as pd
amazon = pd.read_csv('../data/Amazon.csv')
google = pd.read_csv('../data/Google.csv')
apple = pd.read_csv('../data/Apple.csv')
tesla = pd.read_csv('../data/Tesla.csv')
meta = pd.read_csv('../data/Meta.csv')
microsoft = pd.read_csv('../data/Microsoft.csv')
nvidia = pd.read_csv('../data/Nvidia.csv')
amazon['company'] = 'AMZN'
google['company'] = 'GOOGL'
apple['company'] = 'AAPL'
tesla['company'] = 'TSLA'
meta['company'] = 'META'
microsoft['company'] = 'MSFT'
nvidia['company'] = 'NVDA'


In [2]:
#combine all data
df = pd.concat([amazon, google, apple, tesla, meta, microsoft, nvidia], ignore_index=True)
df.head()


,url,authorsByline,articleId,clusterId,source,imageUrl,country,language,pubDate,addDate,...,translation,translatedTitle,translatedDescription,locations,reprint,reprintGroupId,places,people,translatedSummary,company
0,https://www.benzinga.com/markets/options/23/09...,Benzinga Insights,3e0977d3ab0c46a3a63f3c54be31559c,5a7db019e1784220b7f9f1820f7d76de,"{""domain"":""benzinga.com"",""paywall"":false,""loca...",https://cdn.benzinga.com/files/images/story/20...,us,en,2023-09-01T20:18:43+00:00,2023-09-01T20:28:10.586136+00:00,...,NaN,NaN,NaN,[],False,4ee337b9dcbf4525a81b7a5183664bf0,[],[],NaN,AMZN
1,https://www.investors.com/news/technology/amaz...,"PATRICK SEITZ, Investor's Business Daily",d87e2715819b4c768df970877cc3adc9,43fcd4eaf99c4e658f2ea8937bb3b1a1,"{""domain"":""investors.com"",""paywall"":true,""loca...",https://www.investors.com/wp-content/uploads/2...,us,en,2022-08-05T13:57:00+00:00,2022-08-05T14:03:06.437850+00:00,...,NaN,NaN,NaN,[],False,5ecef2bfc08a4829abc39e5bc95487d2,NaN,NaN,NaN,AMZN
2,https://www.benzinga.com/insights/options/24/0...,Benzinga Insights,ccd56c53f85e4048a18d3c5558d24962,4dfd303d36194063a2259979e2f08951,"{""domain"":""benzinga.com"",""paywall"":false,""loca...",https://cdn.benzinga.com/files/images/story/20...,us,en,2024-09-25T17:30:46+00:00,2024-09-25T17:47:54.002068+00:00,...,NaN,NaN,NaN,[],False,86d94c97ebcf4d8ebe50e81f3d83636a,[],[],NaN,AMZN
3,https://www.benzinga.com/insights/news/24/10/4...,Benzinga Insights,3269f58619154c55b08d8b58dc68b4a5,f48106320e3542d196af0d216b604eee,"{""domain"":""benzinga.com"",""paywall"":false,""loca...",https://cdn.benzinga.com/files/images/story/20...,us,en,2024-10-23T15:31:30+00:00,2024-10-25T23:31:30.291124+00:00,...,NaN,NaN,NaN,[],False,dd83e1c061cb4a8eb4013af7bbe6e274,[],[],NaN,AMZN
4,https://www.barrons.com/articles/-amazon-care-...,Eric J. Savitz,078d5e6f599a4ecdbcd64840242c1757,7352bfbd21434aa4bab91cf5eb4514a1,"{""domain"":""barrons.com"",""paywall"":true,""locati...",https://images.barrons.com/im-608344/social,us,en,2022-08-24T23:18:00+00:00,2022-08-24T23:39:20.583059+00:00,...,NaN,NaN,NaN,[],False,bdf340ec49bc4236af501bb87bbb88f3,NaN,NaN,NaN,AMZN


In [3]:
# keep pubdate and summary and sentiment
df = df[['pubDate', 'summary', 'sentiment', 'company']]
df.head()


,pubDate,summary,sentiment,company
0,2023-09-01T20:18:43+00:00,Amazon.com (NASDAQ:AMZN) has taken a bearish s...,"{""positive"":0.21119389,""negative"":0.029195568,...",AMZN
1,2022-08-05T13:57:00+00:00,E-commerce giant Amazon (AMZN) on Friday annou...,"{""positive"":0.2794005,""negative"":0.1272366,""ne...",AMZN
2,2024-09-25T17:30:46+00:00,Amazon.com (NASDAQ:AMZN) has seen unusual acti...,"{""positive"":0.038785376,""negative"":0.7469895,""...",AMZN
3,2024-10-23T15:31:30+00:00,Amazon.com (NASDAQ:AMZN) shares are currently ...,"{""positive"":0.1375702,""negative"":0.45907447,""n...",AMZN
4,2022-08-24T23:18:00+00:00,"Amazon is shutting down Amazon Care, a healthc...","{""positive"":0.10368154,""negative"":0.08483054,""...",AMZN


In [5]:
# trasform pubdate to datetime
df['pubDate'] = pd.to_datetime(df['pubDate'], errors='coerce')
df.head()

# transform pubdate to readable date
df['pubDate'] = df['pubDate'].dt.date
df.head()



,pubDate,summary,sentiment,company
0,2023-09-01,Amazon.com (NASDAQ:AMZN) has taken a bearish s...,"{""positive"":0.21119389,""negative"":0.029195568,...",AMZN
1,2022-08-05,E-commerce giant Amazon (AMZN) on Friday annou...,"{""positive"":0.2794005,""negative"":0.1272366,""ne...",AMZN
2,2024-09-25,Amazon.com (NASDAQ:AMZN) has seen unusual acti...,"{""positive"":0.038785376,""negative"":0.7469895,""...",AMZN
3,2024-10-23,Amazon.com (NASDAQ:AMZN) shares are currently ...,"{""positive"":0.1375702,""negative"":0.45907447,""n...",AMZN
4,2022-08-24,"Amazon is shutting down Amazon Care, a healthc...","{""positive"":0.10368154,""negative"":0.08483054,""...",AMZN


In [6]:
df.info()

df.dropna(inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pubDate    1291 non-null   object
 1   summary    1324 non-null   object
 2   sentiment  1386 non-null   object
 3   company    1400 non-null   object
dtypes: object(4)
memory usage: 43.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 1225 entries, 0 to 1399
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pubDate    1225 non-null   object
 1   summary    1225 non-null   object
 2   sentiment  1225 non-null   object
 3   company    1225 non-null   object
dtypes: object(4)
memory usage: 47.9+ KB


In [7]:
# Function to get the key with the maximum value
import ast
def get_max_key(d):
    return max(d, key=d.get)

# Apply the function to the 'col' column
df['sentiment'] = df['sentiment'].apply(ast.literal_eval)
df['label'] = df['sentiment'].apply(get_max_key)
df.head()




,pubDate,summary,sentiment,company,label
0,2023-09-01,Amazon.com (NASDAQ:AMZN) has taken a bearish s...,"{'positive': 0.21119389, 'negative': 0.0291955...",AMZN,neutral
1,2022-08-05,E-commerce giant Amazon (AMZN) on Friday annou...,"{'positive': 0.2794005, 'negative': 0.1272366,...",AMZN,neutral
2,2024-09-25,Amazon.com (NASDAQ:AMZN) has seen unusual acti...,"{'positive': 0.038785376, 'negative': 0.746989...",AMZN,negative
3,2024-10-23,Amazon.com (NASDAQ:AMZN) shares are currently ...,"{'positive': 0.1375702, 'negative': 0.45907447...",AMZN,negative
4,2022-08-24,"Amazon is shutting down Amazon Care, a healthc...","{'positive': 0.10368154, 'negative': 0.0848305...",AMZN,neutral


In [8]:
def match_label(row):
    sentiment_dict = row['sentiment']
    label = row['label']
    return sentiment_dict.get(label, None)
df['score'] = df.apply(match_label, axis=1)
df.head()

,pubDate,summary,sentiment,company,label,score
0,2023-09-01,Amazon.com (NASDAQ:AMZN) has taken a bearish s...,"{'positive': 0.21119389, 'negative': 0.0291955...",AMZN,neutral,0.759611
1,2022-08-05,E-commerce giant Amazon (AMZN) on Friday annou...,"{'positive': 0.2794005, 'negative': 0.1272366,...",AMZN,neutral,0.593363
2,2024-09-25,Amazon.com (NASDAQ:AMZN) has seen unusual acti...,"{'positive': 0.038785376, 'negative': 0.746989...",AMZN,negative,0.746989
3,2024-10-23,Amazon.com (NASDAQ:AMZN) shares are currently ...,"{'positive': 0.1375702, 'negative': 0.45907447...",AMZN,negative,0.459074
4,2022-08-24,"Amazon is shutting down Amazon Care, a healthc...","{'positive': 0.10368154, 'negative': 0.0848305...",AMZN,neutral,0.811488


In [9]:
# label 0 is negative, 1 is neutral, 2 is positive
df['label'] = df['label'].map({'negative': 0, 'neutral': 1, 'positive': 2})
df.head()



,pubDate,summary,sentiment,company,label,score
0,2023-09-01,Amazon.com (NASDAQ:AMZN) has taken a bearish s...,"{'positive': 0.21119389, 'negative': 0.0291955...",AMZN,1,0.759611
1,2022-08-05,E-commerce giant Amazon (AMZN) on Friday annou...,"{'positive': 0.2794005, 'negative': 0.1272366,...",AMZN,1,0.593363
2,2024-09-25,Amazon.com (NASDAQ:AMZN) has seen unusual acti...,"{'positive': 0.038785376, 'negative': 0.746989...",AMZN,0,0.746989
3,2024-10-23,Amazon.com (NASDAQ:AMZN) shares are currently ...,"{'positive': 0.1375702, 'negative': 0.45907447...",AMZN,0,0.459074
4,2022-08-24,"Amazon is shutting down Amazon Care, a healthc...","{'positive': 0.10368154, 'negative': 0.0848305...",AMZN,1,0.811488


In [10]:
df.drop(columns=['sentiment'], inplace=True)
df.head()

df.to_csv('../data/processed_data_news.csv', index=False)



In [11]:
# grab stock price data
import sys
sys.path.append('/Users/kailiu/StockMarketPrediction-') 

from utils import APIs
aapl = APIs.download_stock_data('AAPL', '1990-01-01', '2024-11-25')
amazon = APIs.download_stock_data('AMZN', '1990-01-01', '2024-11-25')
tesla = APIs.download_stock_data('TSLA', '1990-01-01', '2024-11-25')
meta = APIs.download_stock_data('META', '1990-01-01', '2024-11-25')
google = APIs.download_stock_data('GOOGL', '1990-01-01', '2024-11-25')
microsoft = APIs.download_stock_data('MSFT', '1990-01-01', '2024-11-25')
nvidia = APIs.download_stock_data('NVDA', '1990-01-01', '2024-11-25')
# make data as columns
aapl.reset_index(inplace=True)
amazon.reset_index(inplace=True)
tesla.reset_index(inplace=True)
meta.reset_index(inplace=True)
google.reset_index(inplace=True)
microsoft.reset_index(inplace=True)
nvidia.reset_index(inplace=True)
aapl['company'] = 'AAPL'
amazon['company'] = 'AMZN'
tesla['company'] = 'TSLA'
meta['company'] = 'META'
google['company'] = 'GOOGL'
microsoft['company'] = 'MSFT'
nvidia['company'] = 'NVDA'
# combine all the data
df_stock = pd.concat([aapl, amazon, tesla, meta, google, microsoft, nvidia], ignore_index=True)
df_stock.head()



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume,company
0,1990-01-02,0.262129,0.332589,0.334821,0.312500,0.314732,183198400,AAPL
1,1990-01-03,0.263889,0.334821,0.339286,0.334821,0.339286,207995200,AAPL
2,1990-01-04,0.264769,0.335938,0.345982,0.332589,0.341518,221513600,AAPL
3,1990-01-05,0.265648,0.337054,0.341518,0.330357,0.337054,123312000,AAPL
4,1990-01-08,0.267408,0.339286,0.339286,0.330357,0.334821,101572800,AAPL


In [12]:
df_news = pd.read_csv('../data/processed_data_news.csv')
df_news.head()



,pubDate,summary,company,label,score
0,2023-09-01,Amazon.com (NASDAQ:AMZN) has taken a bearish s...,AMZN,1,0.759611
1,2022-08-05,E-commerce giant Amazon (AMZN) on Friday annou...,AMZN,1,0.593363
2,2024-09-25,Amazon.com (NASDAQ:AMZN) has seen unusual acti...,AMZN,0,0.746989
3,2024-10-23,Amazon.com (NASDAQ:AMZN) shares are currently ...,AMZN,0,0.459074
4,2022-08-24,"Amazon is shutting down Amazon Care, a healthc...",AMZN,1,0.811488


In [13]:
# combine news and stock data by company and date(pubDate)
# Convert both date columns to datetime
df_news['pubDate'] = pd.to_datetime(df_news['pubDate'])
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df = pd.merge(df_news, df_stock, left_on=['company', 'pubDate'], right_on=['company', 'Date'], how='right')
df.drop(columns=['pubDate'], inplace=True)
df.head()




,summary,company,label,score,Date,Adj Close,Close,High,Low,Open,Volume
0,NaN,AAPL,NaN,NaN,1990-01-02,0.262129,0.332589,0.334821,0.312500,0.314732,183198400
1,NaN,AAPL,NaN,NaN,1990-01-03,0.263889,0.334821,0.339286,0.334821,0.339286,207995200
2,NaN,AAPL,NaN,NaN,1990-01-04,0.264769,0.335938,0.345982,0.332589,0.341518,221513600
3,NaN,AAPL,NaN,NaN,1990-01-05,0.265648,0.337054,0.341518,0.330357,0.337054,123312000
4,NaN,AAPL,NaN,NaN,1990-01-08,0.267408,0.339286,0.339286,0.330357,0.334821,101572800


In [14]:
df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43104 entries, 0 to 43103
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   summary    1143 non-null   object        
 1   company    43104 non-null  object        
 2   label      1143 non-null   float64       
 3   score      1143 non-null   float64       
 4   Date       43104 non-null  datetime64[ns]
 5   Adj Close  43104 non-null  float64       
 6   Close      43104 non-null  float64       
 7   High       43104 non-null  float64       
 8   Low        43104 non-null  float64       
 9   Open       43104 non-null  float64       
 10  Volume     43104 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(1), object(2)
memory usage: 3.6+ MB


In [15]:
# 计算涨跌幅
df['change'] = (df['Close'] - df['Open'])/df['Open'] * 100
df.head()



,summary,company,label,score,Date,Adj Close,Close,High,Low,Open,Volume,change
0,NaN,AAPL,NaN,NaN,1990-01-02,0.262129,0.332589,0.334821,0.312500,0.314732,183198400,5.673721
1,NaN,AAPL,NaN,NaN,1990-01-03,0.263889,0.334821,0.339286,0.334821,0.339286,207995200,-1.316003
2,NaN,AAPL,NaN,NaN,1990-01-04,0.264769,0.335938,0.345982,0.332589,0.341518,221513600,-1.633884
3,NaN,AAPL,NaN,NaN,1990-01-05,0.265648,0.337054,0.341518,0.330357,0.337054,123312000,0.000000
4,NaN,AAPL,NaN,NaN,1990-01-08,0.267408,0.339286,0.339286,0.330357,0.334821,101572800,1.333553


In [17]:
data = df[df['summary'].notna()]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1143 entries, 7885 to 43096
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   summary    1143 non-null   object        
 1   company    1143 non-null   object        
 2   label      1143 non-null   float64       
 3   score      1143 non-null   float64       
 4   Date       1143 non-null   datetime64[ns]
 5   Adj Close  1143 non-null   float64       
 6   Close      1143 non-null   float64       
 7   High       1143 non-null   float64       
 8   Low        1143 non-null   float64       
 9   Open       1143 non-null   float64       
 10  Volume     1143 non-null   int64         
 11  change     1143 non-null   float64       
dtypes: datetime64[ns](1), float64(8), int64(1), object(2)
memory usage: 116.1+ KB


In [20]:
data.sort_values(by='Date', ascending=False).head(-1)

,summary,company,label,score,Date,Adj Close,Close,High,Low,Open,Volume,change
19438,Investors in Tesla (NASDAQ:TSLA) have been tak...,TSLA,1.0,0.487321,2024-11-22,352.559998,352.559998,361.529999,337.700012,341.089996,89140700,3.362749
22632,"Fred Alger Management, an investment managemen...",META,2.0,0.875961,2024-11-21,563.090027,563.090027,570.000000,549.049988,569.520020,11154700,-1.129020
8824,Apple (AAPL) has begun selling advertising spa...,AAPL,2.0,0.559948,2024-11-19,228.279999,228.279999,230.160004,226.660004,226.979996,36211800,0.572739
19434,Tesla Inc shares have risen following reports ...,TSLA,0.0,0.628493,2024-11-18,338.739990,338.739990,348.549988,330.010010,340.730011,126547500,-0.584046
22629,The dollar has strengthened in 2024 due to the...,META,2.0,0.822451,2024-11-18,554.400024,554.400024,559.900024,550.090027,557.039978,14346700,-0.473925
...,...,...,...,...,...,...,...,...,...,...,...,...
7954,Analysts expected Apple earnings of $1.01 a sh...,AAPL,0.0,0.864846,2021-07-28,142.222443,144.979996,146.970001,142.539993,144.809998,118931200,0.117394
35689,Software powerhouse Microsoft (MSFT) on Thursd...,MSFT,2.0,0.551069,2021-06-24,258.977722,266.690002,267.850006,265.470001,266.160004,21446900,0.199128
7924,That cool smart device on your wrist will soon...,AAPL,2.0,0.910649,2021-06-15,127.174217,129.639999,130.600006,129.389999,129.940002,62746300,-0.230878
7885,"However, Apple stock fell on Tuesday. \n\nOn t...",AAPL,0.0,0.818085,2021-04-20,130.356781,133.110001,135.529999,131.809998,135.020004,94812300,-1.414608


In [16]:
df.to_csv('../data/processed_data_news_stock.csv', index=False)